In [1]:
from cpsat_autotune import (
    tune_for_quality_within_timelimit,
    import_model,
    tune_time_to_optimal,
)

In [2]:
model = import_model("models/model_proto_-1368620059351071685.pb")

In [3]:
# tune_for_quality_within_timelimit(models[0], 3, 10, "minimize", n_samples_per_param=2, max_samples_per_param=5, n_trials=100)
best = tune_time_to_optimal(
    model,
    timelimit_in_s=6,  # Enter a time limit slightly above what the solver with default parameters needs
    n_samples_per_param=5,  # The higher the number, the more accurate the tuning, but the longer it takes
    max_samples_per_param=10,  # After this number of samples for a parameter set, we just take the average from the history
    n_trials=50,  # Number of trials to run with Optuna
)

[I 2024-08-16 11:16:54,966] A new study created in memory with name: no-name-3d85195b-1b00-4a4e-94a7-c2bbaa7a4cca


Computing baseline


[I 2024-08-16 11:17:10,302] Trial 0 finished with value: -1.5331676 and parameters: {'preferred_variable_order': 0, 'use_erwa_heuristic': False, 'also_bump_variables_in_conflict_reasons': False, 'binary_minimization_algorithm': 1, 'clause_cleanup_protection': 0, 'presolve_bve_threshold': 1, 'max_presolve_iterations': 2, 'cp_model_probing_level': 2, 'presolve_probing_deterministic_time_limit': 4, 'encode_complex_linear_constraint_with_integer': False, 'ignore_subsolvers:default_lp': False, 'ignore_subsolvers:fixed': False, 'ignore_subsolvers:no_lp': False, 'ignore_subsolvers:max_lp': False, 'ignore_subsolvers:pseudo_costs': False, 'ignore_subsolvers:reduced_costs': False, 'ignore_subsolvers:quick_restart': False, 'ignore_subsolvers:quick_restart_no_lp': False, 'ignore_subsolvers:lb_tree_search': False, 'ignore_subsolvers:probing': False, 'search_branching': 0, 'repair_hint': False, 'use_lb_relax_lns': False, 'use_objective_lb_search': False, 'use_objective_shaving_search': False, 'optim

Baseline: [-2.013006, -1.605162, -1.517007, -1.320981, -2.255245, -1.102053, -1.271127, -1.122326, -1.74061, -1.384159]


[I 2024-08-16 11:17:16,325] Trial 1 finished with value: -60.0 and parameters: {'preferred_variable_order': 1, 'use_erwa_heuristic': True, 'also_bump_variables_in_conflict_reasons': True, 'binary_minimization_algorithm': 4, 'clause_cleanup_protection': 1, 'presolve_bve_threshold': 2, 'max_presolve_iterations': 4, 'cp_model_probing_level': 0, 'presolve_probing_deterministic_time_limit': 2, 'encode_complex_linear_constraint_with_integer': True, 'ignore_subsolvers:default_lp': False, 'ignore_subsolvers:fixed': True, 'ignore_subsolvers:no_lp': False, 'ignore_subsolvers:max_lp': True, 'ignore_subsolvers:pseudo_costs': True, 'ignore_subsolvers:reduced_costs': True, 'ignore_subsolvers:quick_restart': True, 'ignore_subsolvers:quick_restart_no_lp': True, 'ignore_subsolvers:lb_tree_search': True, 'ignore_subsolvers:probing': False, 'search_branching': 1, 'repair_hint': True, 'use_lb_relax_lns': True, 'use_objective_lb_search': True, 'use_objective_shaving_search': False, 'optimize_with_core': Fa

Parameter Statistics:
	presolve_bve_threshold: 1000
	max_presolve_iterations: 1
	presolve_probing_deterministic_time_limit: 5.0
	symmetry_level: 0
	cp_model_probing_level: 0
	use_objective_lb_search: True
	feasibility_jump_linearization_level: 0
	cut_level: 0
	search_branching: 2
	clause_cleanup_protection: 1
	ignore_subsolvers: ('fixed', 'lb_tree_search', 'quick_restart')
	fp_rounding: 3
	preferred_variable_order: 1
	binary_minimization_algorithm: 4
 Mean: 0.44042400000000004
 Median: 0.403767
 Standard Deviation: 0.09656962660588472
 Max: 0.5603
 Min: 0.33805
 Count: 5

Parameter Statistics:
	cut_level: 2
	add_objective_cut: True
	binary_minimization_algorithm: 0
	fp_rounding: 1
	presolve_probing_deterministic_time_limit: 10.0
	max_presolve_iterations: 2
	max_all_diff_cut_size: 32
	ignore_subsolvers: ('quick_restart_no_lp',)
 Mean: 1.2887272
 Median: 1.144031
 Standard Deviation: 0.345820992139228
 Max: 1.887978
 Min: 0.878809
 Count: 5

Parameter Statistics:
	symmetry_level: 1
	cp_m

In [3]:
best = tune_for_quality_within_timelimit(
    model,
    timelimit_in_s=0.5,
    obj_for_timeout=0,
    direction="maximize",
    n_samples_per_param=5,
    max_samples_per_param=10,
    n_trials=50,
)

[I 2024-08-16 11:29:55,652] A new study created in memory with name: no-name-e4b187ce-7135-4442-a778-a6ffcc5136f5


Computing baseline


[I 2024-08-16 11:30:00,827] Trial 0 finished with value: 131648.4 and parameters: {'preferred_variable_order': 0, 'use_erwa_heuristic': False, 'also_bump_variables_in_conflict_reasons': False, 'binary_minimization_algorithm': 1, 'clause_cleanup_protection': 0, 'presolve_bve_threshold': 1, 'max_presolve_iterations': 2, 'cp_model_probing_level': 2, 'presolve_probing_deterministic_time_limit': 4, 'encode_complex_linear_constraint_with_integer': False, 'ignore_subsolvers:default_lp': False, 'ignore_subsolvers:fixed': False, 'ignore_subsolvers:no_lp': False, 'ignore_subsolvers:max_lp': False, 'ignore_subsolvers:pseudo_costs': False, 'ignore_subsolvers:reduced_costs': False, 'ignore_subsolvers:quick_restart': False, 'ignore_subsolvers:quick_restart_no_lp': False, 'ignore_subsolvers:lb_tree_search': False, 'ignore_subsolvers:probing': False, 'search_branching': 0, 'repair_hint': False, 'use_lns_only': False, 'use_lb_relax_lns': False, 'use_objective_lb_search': False, 'use_objective_shaving_s

Baseline: [130158.0, 130418.0, 132805.0, 132509.0, 132359.0, 131215.0, 131124.0, 130671.0, 132805.0, 132420.0]


[I 2024-08-16 11:30:02,882] Trial 1 finished with value: 128143.0 and parameters: {'preferred_variable_order': 1, 'use_erwa_heuristic': True, 'also_bump_variables_in_conflict_reasons': False, 'binary_minimization_algorithm': 0, 'clause_cleanup_protection': 1, 'presolve_bve_threshold': 2, 'max_presolve_iterations': 3, 'cp_model_probing_level': 1, 'presolve_probing_deterministic_time_limit': 2, 'encode_complex_linear_constraint_with_integer': True, 'ignore_subsolvers:default_lp': True, 'ignore_subsolvers:fixed': False, 'ignore_subsolvers:no_lp': True, 'ignore_subsolvers:max_lp': True, 'ignore_subsolvers:pseudo_costs': True, 'ignore_subsolvers:reduced_costs': False, 'ignore_subsolvers:quick_restart': False, 'ignore_subsolvers:quick_restart_no_lp': False, 'ignore_subsolvers:lb_tree_search': True, 'ignore_subsolvers:probing': True, 'search_branching': 3, 'repair_hint': True, 'use_lns_only': True, 'use_lb_relax_lns': True, 'use_objective_lb_search': True, 'use_objective_shaving_search': Fals

Parameter Statistics:
	fp_rounding: 0
	diversify_lns_params: True
	max_presolve_iterations: 1
	presolve_bve_threshold: 1000
	clause_cleanup_protection: 2
	binary_minimization_algorithm: 3
	polish_lp_solution: True
	use_erwa_heuristic: True
	search_branching: 6
	ignore_subsolvers: ('lb_tree_search', 'probing', 'quick_restart_no_lp')
	repair_hint: True
	use_lb_relax_lns: True
	cp_model_probing_level: 1
	use_lns_only: True
	presolve_probing_deterministic_time_limit: 1.0
 Mean: 132805.0
 Median: 132805.0
 Standard Deviation: 0.0
 Max: 132805.0
 Min: 132805.0
 Count: 5

Parameter Statistics:
	symmetry_level: 1
	fp_rounding: 0
	max_presolve_iterations: 1
	presolve_bve_threshold: 1000
	polish_lp_solution: True
	encode_complex_linear_constraint_with_integer: True
	presolve_probing_deterministic_time_limit: 5.0
	ignore_subsolvers: ('fixed',)
	repair_hint: True
	search_branching: 2
 Mean: 132083.2
 Median: 132359.0
 Standard Deviation: 913.3355133793933
 Max: 132805.0
 Min: 130305.0
 Count: 5

P